In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from  sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

train=pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
train = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_price':'mean','item_cnt_day':'sum'}).reset_index()
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
Id=test['ID']
test['date_block_num'] = 0
test=test[['date_block_num','shop_id','item_id']]

item_price=dict(train.groupby('item_id')['item_price'].last().reset_index().values)
test['item_price']=test.item_id.map(item_price)
test['item_price']=test['item_price'].fillna(test['item_price'].median())
# print(test.head())

X=train.drop(['item_cnt_day'],axis=1)
y=train['item_cnt_day']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=44,shuffle=True)
lrm=RandomForestRegressor(n_estimators=100,max_depth=2, random_state=33)

lrm.fit(X_train,y_train)
score=lrm.score(X_train,y_train)
print(score)
y_pred=lrm.predict(X_test)
mae=mean_squared_error(y_test,y_pred)
print(mae)

y_pred1=lrm.predict(test)

output = pd.DataFrame({'Id':Id,'item_cnt_month': y_pred1})
output.to_csv('Mysubmission.csv', index=False)
